# Welcome to the Math Test! Please run the cell below to begin :)

In [39]:
from finalmathtest import run_math

In [42]:
# RUN THIS CELL TO RUN THE TEST :)
run_math()

Box(children=(Text(value='', description='Answer: ', style=DescriptionStyle(description_width='initial')),), l…